In [1]:
print("hello world")

hello world


# Flask Error Handling and Debugging Guide

This guide covers how to effectively debug Flask applications and implement user-friendly error handling.



Table of Contents

    Flask Debugger

    Common Error Types

    Custom Error Pages

    Debugging Best Practices

    Complete Example Application

### Flask Debugger

#### Enabling Debug Mode


In [ ]:
from flask import Flask

app = Flask(__name__)
app.debug = True  # Enable debug mode
# OR
app.run(debug=True)

## Development Configuration

In [2]:
import os
from flask import Flask

app = Flask(__name__)

# Configuration
class Config:
    DEBUG = False
    TESTING = False

class DevelopmentConfig(Config):
    DEBUG = True
    SECRET_KEY = 'dev-secret-key'

class ProductionConfig(Config):
    DEBUG = False
    SECRET_KEY = os.environ.get('SECRET_KEY', 'prod-secret-key')

# Choose configuration based on environment
if os.environ.get('FLASK_ENV') == 'development':
    app.config.from_object(DevelopmentConfig)
else:
    app.config.from_object(ProductionConfig)

### Common Error Types


HTTP Status Codes

In [3]:
from flask import Flask, abort

app = Flask(__name__)

# Common HTTP Errors
@app.route('/examples')
def error_examples():
    # 400 - Bad Request
    # 401 - Unauthorized
    # 403 - Forbidden
    # 404 - Not Found
    # 405 - Method Not Allowed
    # 500 - Internal Server Error
    
    # Manually trigger errors
    abort(404)  # Page not found
    abort(500)  # Internal server error

## Custom Error Classes 

In [4]:
class DatabaseConnectionError(Exception):
    """Custom exception for database issues"""
    pass

class ValidationError(Exception):
    """Custom exception for data validation"""
    pass

# Register custom exceptions
@app.errorhandler(DatabaseConnectionError)
def handle_database_error(error):
    return "Database connection failed", 503

@app.errorhandler(ValidationError)
def handle_validation_error(error):
    return "Invalid data provided", 400

## Custom Error Pages

Basic Error Handlers

In [5]:
from flask import Flask, render_template

app = Flask(__name__)

@app.errorhandler(404)
def page_not_found(error):
    return render_template('404.html'), 404

@app.errorhandler(500)
def internal_server_error(error):
    return render_template('500.html'), 500

@app.errorhandler(403)
def forbidden(error):
    return render_template('403.html'), 403

## Advanced Error Handling

In [6]:
import logging
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

def log_error(error, status_code):
    """Log errors with context information"""
    app.logger.error(
        f'Error {status_code}: {error}\n'
        f'Path: {request.path}\n'
        f'Method: {request.method}\n'
        f'User Agent: {request.user_agent}\n'
        f'Referrer: {request.referrer}'
    )

@app.errorhandler(404)
def not_found_error(error):
    log_error(error, 404)
    
    if request.accept_mimetypes.accept_json and \
       not request.accept_mimetypes.accept_html:
        return jsonify({'error': 'Resource not found'}), 404
    
    return render_template('errors/404.html'), 404

@app.errorhandler(500)
def internal_error(error):
    log_error(error, 500)
    
    # In production, you might want to notify admins
    if not app.debug:
        # Send email to admin, log to monitoring service, etc.
        pass
    
    if request.accept_mimetypes.accept_json and \
       not request.accept_mimetypes.accept_html:
        return jsonify({'error': 'Internal server error'}), 500
    
    return render_template('errors/500.html'), 500

# Handle all exceptions
@app.errorhandler(Exception)
def handle_unexpected_error(error):
    log_error(error, 500)
    
    if not app.debug:
        # In production, provide generic error message
        if request.accept_mimetypes.accept_json and \
           not request.accept_mimetypes.accept_html:
            return jsonify({'error': 'An unexpected error occurred'}), 500
        
        return render_template('errors/500.html'), 500
    
    # In development, let Flask debugger handle it
    return error

## Debugging Best Practices
Logging Configuration

In [7]:
import logging
from logging.handlers import RotatingFileHandler
import os

def setup_logging(app):
    """Configure application logging"""
    
    if not app.debug:
        # Production logging
        if not os.path.exists('logs'):
            os.mkdir('logs')
        
        file_handler = RotatingFileHandler(
            'logs/flask_app.log', 
            maxBytes=10240, 
            backupCount=10
        )
        file_handler.setFormatter(logging.Formatter(
            '%(asctime)s %(levelname)s: %(message)s '
            '[in %(pathname)s:%(lineno)d]'
        ))
        file_handler.setLevel(logging.INFO)
        app.logger.addHandler(file_handler)
        app.logger.setLevel(logging.INFO)
        app.logger.info('Flask application startup')
    
    else:
        # Development logging
        app.logger.setLevel(logging.DEBUG)

# Initialize logging
setup_logging(app)

[2025-10-03 23:12:31,755] INFO in 2421739839: Flask application startup


## Debugging Utilities

In [8]:
from flask import current_app
import functools

def debug_route(f):
    """Decorator to add debug information to routes"""
    @functools.wraps(f)
    def decorated_function(*args, **kwargs):
        current_app.logger.debug(
            f"Route called: {request.path} "
            f"with method: {request.method}"
        )
        try:
            return f(*args, **kwargs)
        except Exception as e:
            current_app.logger.error(
                f"Error in route {request.path}: {str(e)}"
            )
            raise
    return decorated_function

@app.route('/debug-info')
@debug_route
def debug_info():
    """Route with debug information"""
    debug_data = {
        'flask_debug': current_app.debug,
        'environment': os.environ.get('FLASK_ENV', 'production'),
        'secret_key_configured': bool(current_app.secret_key),
    }
    return jsonify(debug_data)

### Complete Example Application

Here's a complete Flask application with comprehensive error handling:
Project Structure

flask_app/
├── app.py
├── templates/
│   ├── base.html
│   ├── index.html
│   └── errors/
│       ├── 400.html
│       ├── 403.html
│       ├── 404.html
│       └── 500.html
├── static/
│   └── css/
│       └── style.css
└── logs/